### Loading the required libraries

In [21]:
from dotenv import load_dotenv
import os
import pandas as pd
from web3 import Web3
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import seaborn as sns

### Extracting the Data (Blocks)

In [22]:
# loading environment variable
load_dotenv()

# Getting the infura API key
infura_api_key = os.getenv('INFURA_API_KEY')

# Connect to Infura's Ethereum node
# infura_url = f"https://mainnet.infura.io/v3/{infura_api_key}"
infura_url = f"https://mainnet.infura.io/v3/9806016edf084f199f87c6365c8ac03b"
w3 = Web3(Web3.HTTPProvider(infura_url))


In [23]:
# Function to fetch block data
def Fetch_block_data(start_block, end_block):
    block_data = []
    for block_num in range(start_block, end_block + 1):
        block = w3.eth.get_block(block_num)
        block_info = {
            "number": block['number'],
            "timestamp": block['timestamp'],
            "transactions": len(block['transactions']),
            "gas_used": block['gasUsed'],
            "gas_limit": block['gasLimit'],
            "size": block['size']
        }
        block_data.append(block_info)
    return pd.DataFrame(block_data)

# Fetch data for the last 1000 blocks
latest_block = w3.eth.block_number
start_block = latest_block - 1000
block_df = Fetch_block_data(start_block, latest_block)

# Save block data to CSV
block_df.to_csv('data/blocks.csv', index=False)

In [24]:
block_df = pd.read_csv("C:/Users/39351/github/Web3_data_analysis/data/blocks.csv")

### Plot The Number of Transactions Over Time

In [25]:
# Create the line plot using Plotly Express
fig = px.line(block_df, x='timestamp', y='transactions', title='Number of Transactions Over Time')

# Update the layout to include axis labels and rotate the x-axis ticks
fig.update_layout(
    xaxis_title='Timestamp',
    yaxis_title='Number of Transactions',
    xaxis=dict(tickangle=45)
)


# Display the plot
fig.show()


### Plot the Gas Used Over Time

In [26]:
fig2 = px.line(block_df, x = 'timestamp', y = 'gas_used', title = 'Gas used over Time')

fig2.update_layout(
    xaxis_title ='Timestamp',
    yaxis_title = 'Gas Used',
    xaxis = dict(tickangle=45))

fig2.update_traces(line=dict(color='Orange'))

fig2.show()

### Distribution of Transaction per Block

In [27]:

fig3 = make_subplots(rows=2, cols=1, subplot_titles=(
    "Histogram of Transactions per Block",
    "Distribution Plot of Transactions per Block"
))

# 1. Histogram
hist_fig = px.histogram(block_df, x='transactions', nbins=30)
for trace in hist_fig['data']:
    fig3.add_trace(trace, row=1, col=1)

# 2. Distribution plot
dist_fig = ff.create_distplot([block_df['transactions']], group_labels=['Transactions'], colors=['purple'])
for trace in dist_fig['data']:
    fig3.add_trace(trace, row=2, col=1)

# Update layout
fig3.update_layout(
    height=700,
    title_text="Transaction Distribution Across Blocks",
    showlegend=False
)

fig3.update_xaxes(title_text='Number of Transactions', row=1, col=1)
fig3.update_yaxes(title_text='Frequency', row=1, col=1)
fig3.update_xaxes(title_text='Number of Transactions', row=2, col=1)
fig3.update_yaxes(title_text='Density', row=2, col=1)

fig3.show()



### Fetch the Transaction Data

In [28]:
# Fetch data for the last 1000 blocks
latest_block = w3.eth.block_number
start_block = latest_block - 1000

# Function to fetch transaction data for a specific block
def Fetch_transaction_data(block_number):
    block = w3.eth.get_block(block_number, full_transactions=True)
    transaction_data = []
    for tx in block.transactions:
        tx_info = {
            "block_number": tx['blockNumber'],
            "from": tx['from'],
            "to": tx['to'],
            "value": w3.from_wei(tx['value'], 'ether'),
            "gas": tx['gas'],
            "gas_price": w3.from_wei(tx['gasPrice'], 'gwei'),
            "hash": tx['hash'].hex()
        }
        transaction_data.append(tx_info)
    return pd.DataFrame(transaction_data)

# Fetch transaction data for the last block
transaction_df = Fetch_transaction_data(latest_block)

# Save transaction data to CSV
transaction_df.to_csv('data/transactions.csv', index=False)

In [29]:
# Summary statistics of transaction values
transaction_summary = transaction_df['value'].describe()
print(transaction_summary)

count     240
unique    112
top         0
freq      121
Name: value, dtype: int64


In [34]:

fig4 = px.histogram(transaction_df, x= 'value', nbins=30, title = 'Distribution of Transactions Values')

fig4.update_layout(
    xaxis_title='Value (Ether)',
    yaxis_title='Frequency',
)


fig4.show()


In [35]:
transaction_df.head()

,block_number,from,to,value,gas,gas_price,hash
0,23033307,0x9e65DD108846E7fbBc7E8313dD8A094586E3c22b,0x3328F7f4A1D1C57c35df56bBf0c9dCAFCA309C49,0,481617,102.579285013,0xd05b2e897040898712b656538be94622c21dda21707e...
1,23033307,0xa8DFB8Cc7f9843c3E7BeC636BD08c3487b72dC40,0x5DDf30555EE9545c8982626B7E3B6F70e5c2635f,0.041,337508,32.579285013,0xa19585d1d90bc1a7d95310a6a628c084e293a7db8c35...
2,23033307,0x7830c87C02e56AFf27FA8Ab1241711331FA86F43,0xA9D1e08C7793af67e9d92fe308d5697FB81d3E43,0,2000000,3.579285013,0x2fbd62db1234df7ef3b891109d0e7ebe2a05844e8f71...
3,23033307,0x7830c87C02e56AFf27FA8Ab1241711331FA86F43,0xA9D1e08C7793af67e9d92fe308d5697FB81d3E43,0,2000000,3.579285013,0x1e0dfc93c1a127885412c8f2773bff33efc831f6c8b8...
4,23033307,0x1740000006ab849AdF993a812290606dF21C6b59,0x6a0D036796Cf16a84c92a2a205D45104494DDc80,3.33E-16,2100000,2.609285013,0x68c322fdba4f59b709a9e8a846835c41d4d5e9e358c4...
